<img src="img/mCIDaeNnb.png" alt="Logo CiDAEN" align="right">




<br><br><br>
<h2><font color="#00586D" size=4>Módulo 7</font></h2>



<h1><font color="#00586D" size=5>Capstone 7. Parte 2: Pronóstico de demanda eléctrica</font></h1>

<br><br><br>
<div style="text-align: right">
<font color="#00586D" size=3>Enrique González, Luis de la Ossa</font><br>
<font color="#00586D" size=3>Máster en Ciencia de Datos e Ingeniería de Datos en la Nube I </font><br>
<font color="#00586D" size=3>Universidad de Castilla-La Mancha</font>

</div>

---

<a id="indice"></a>
<h2><font color="#00586D" size=5>Índice</font></h2>


* [1. Creación del modelo de pronóstico](#section1)
* [2. Ingesta y visualización del pronóstico](#section2)

In [8]:
import matplotlib.pyplot as plt
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error

# Optimiza los gráficos para pantalla retina
%config InlineBackend.figure_format = 'retina'
# Por defecto usamos el backend inline
%matplotlib inline

# Oculta warnings
import warnings
warnings.simplefilter('ignore')

# La libreta ocupa así el 95% de la pantalla
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

En la siguiente parte del capstone crearéis un modelo de pronóstico a partir de los datos ingestados en InfluxDB e añadiréis el pronóstico a un día vista de vuelta a InfluxDB de forma que en el sistema prototipado se pueda ver el pronóstico del día. Específicamente, para la creación del modelo utilizaréis los datos hasta el último día completo de datos y el pronóstico incluirá el dia siguiente (que podrá tener datos o no en nuestro sistema, pero que no los usaremos para entrenar). 

---

<a id="section1"></a>
## <font color="#00586D"> 1. Creación del modelo de pronóstico</font>
<br>

Para la creación del modelo de pronóstico, necesitaremos descargarnos los datos de nuestra instancia para poder tener datos con los que entrenar. 


--- 

### <font color="#004D7F"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#004D7F"></i> Tarea 1</font>
<br>

Implementa la siguiente función que devuelve un DataFrame con todos los datos disponibles de demanda eléctrica almacenados en tu instancia de InfluxDB. Solo nos interesa la serie de demanda real, por lo que tendréis que filtrar el Forecast de red eléctrica usando Flux. 

Necesitaréis añadir la información de autenticación para vuestra instancia de InfluxDB a continuación. 

_Pista_: La práctica de InfluxDB que hicimos en este módulo os resultará útil para poder formar la consulta. 

In [12]:
token = "mcidaen_token"
org = "mcidaen"
bucket = "capstone7"

client = InfluxDBClient(url="http://influxdb:8086", token=token)

In [ ]:
def load_from_influxdb(client: InfluxDBClient, bucket: str, org: str) -> pd.DataFrame:
    """
    Escribe el DataFrame en InfluxDB
    """
    
    ######## Solución: Guarda el resultado de la consulta en df

    ###
    
    df_query = df.loc[:, ['_time', '_value']]
    df_query['datetime'] = pd.to_datetime(df_query['_time'], utc=True).dt.tz_localize(None)
    df_query['RealDemand'] = df_query['_value']
    df_query = df_query[['datetime', 'RealDemand']]
    df_query.index = df_query['datetime']
    return df_query['RealDemand'].sort_index()

In [19]:
df = load_from_influxdb(client, bucket, org)
df['2022'].sample(20, random_state=0)

datetime
2022-08-17 13:00:00    28558.083333
2022-03-22 20:00:00    32101.333333
2022-06-18 01:00:00    24592.250000
2022-02-23 22:00:00    27416.500000
2022-04-27 13:00:00    29158.500000
2022-03-29 07:00:00    30680.500000
2022-06-07 00:00:00    22885.583333
2022-03-10 12:00:00    31206.666667
2022-09-25 05:00:00    19363.166667
2022-08-19 17:00:00    28068.416667
2022-11-06 04:00:00    17734.416667
2022-08-19 05:00:00    23990.666667
2022-02-22 22:00:00    27137.000000
2022-10-19 03:00:00    21814.916667
2022-12-21 01:00:00    21955.166667
2022-04-28 04:00:00    24237.833333
2022-12-01 21:00:00    29663.833333
2022-07-19 18:00:00    33729.666667
2022-07-03 06:00:00    21313.833333
2022-02-19 00:00:00    24770.666667
Name: RealDemand, dtype: float64

Resultado esperado:

| datetime            |   RealDemand |
|:--------------------|-------------:|
| 2022-08-17 13:00:00 |      28558.1 |
| 2022-03-22 20:00:00 |      32101.3 |
| 2022-06-18 01:00:00 |      24592.2 |
| 2022-02-23 22:00:00 |      27416.5 |
| 2022-04-27 13:00:00 |      29158.5 |
| 2022-03-29 07:00:00 |      30680.5 |
| 2022-06-07 00:00:00 |      22885.6 |
| 2022-03-10 12:00:00 |      31206.7 |
| 2022-09-25 05:00:00 |      19363.2 |
| 2022-08-19 17:00:00 |      28068.4 |
| 2022-11-06 04:00:00 |      17734.4 |
| 2022-08-19 05:00:00 |      23990.7 |
| 2022-02-22 22:00:00 |      27137   |
| 2022-10-19 03:00:00 |      21814.9 |
| 2022-12-21 01:00:00 |      21955.2 |
| 2022-04-28 04:00:00 |      24237.8 |
| 2022-12-01 21:00:00 |      29663.8 |
| 2022-07-19 18:00:00 |      33729.7 |
| 2022-07-03 06:00:00 |      21313.8 |
| 2022-02-19 00:00:00 |      24770.7 |


<div style="text-align: right"><font size=4> <i class="fa fa-check-square-o" aria-hidden="true" style="color:#00586D"></i></font></div>

--- 

Con estos datos, el objetivo es crear un modelo de forecasting que nos permita pronosticar a un día vista la demanda de energía horaria. Aunque podéis usar el modelo que estiméis (siempre que funcione con el resto de componentes) en este caso hemos optado por crear un modelo de Prophet. 

Para facilitar su integración con el resto de código y poder ejecutar el modelo para pronosticar varios días, crearemos una función que entrene el modelo y realice un pronóstico para un conjunto de test. Para ello crearemos un modelo capaz de pronosticar, dado un histórico, la demanda de energía horaria del día siguiente al histórico. 


--- 

### <font color="#004D7F"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#004D7F"></i> Tarea 2</font>
<br>

Implementa la siguiente función que, dada la serie temporal de demanda anterior, _training_ y una lista de timestamps, _test_:
- Entrena un modelo de Prophet con los datos de _training_
- Pronostica las horas pasadas en _test_ 

Puedes asumir que el conjunto de datos tiene el formato de salida de la función de la tarea anterior.

Opcional: Como trabajo opcional, puedes probar varios modelos de los expuestos durante el módulo para ver si puedes obtener un mejor resultado. Esto se tendrá en cuenta a la hora de evaluación del capstone. 

In [21]:
import datetime
from prophet import Prophet

def train_forecast(training: pd.DataFrame, test: list) -> pd.DataFrame:
    """
    Entrena con training y pronostica los datos de test
    """
    
    ######## Solución

    #########

In [25]:
test = list(pd.date_range(start=datetime.datetime(2023, 2, 24), freq='H', periods=24))

In [27]:
result = train_forecast(df[:datetime.datetime(2023, 2, 24)], test)

13:24:12 - cmdstanpy - INFO - Chain [1] start processing
13:24:19 - cmdstanpy - INFO - Chain [1] done processing


In [29]:
result

datetime
2023-02-24 00:00:00    26203.955169
2023-02-24 01:00:00    25438.531198
2023-02-24 02:00:00    25002.658036
2023-02-24 03:00:00    25110.605144
2023-02-24 04:00:00    26045.391911
2023-02-24 05:00:00    27773.144263
2023-02-24 06:00:00    29804.732413
2023-02-24 07:00:00    31483.929592
2023-02-24 08:00:00    32447.041337
2023-02-24 09:00:00    32818.973247
2023-02-24 10:00:00    32970.426706
2023-02-24 11:00:00    33098.528803
2023-02-24 12:00:00    33063.951347
2023-02-24 13:00:00    32644.826011
2023-02-24 14:00:00    31925.584272
2023-02-24 15:00:00    31379.332628
2023-02-24 16:00:00    31499.003632
2023-02-24 17:00:00    32298.551500
2023-02-24 18:00:00    33173.658905
2023-02-24 19:00:00    33299.901735
2023-02-24 20:00:00    32245.993259
2023-02-24 21:00:00    30273.349091
2023-02-24 22:00:00    28081.696255
2023-02-24 23:00:00    26265.178496
Name: yhat, dtype: float64

Resultado esperado (aprox.):
    
| datetime            |    yhat |
|:--------------------|--------:|
| 2023-02-24 00:00:00 | 26204   |
| 2023-02-24 01:00:00 | 25438.5 |
| 2023-02-24 02:00:00 | 25002.7 |
| 2023-02-24 03:00:00 | 25110.6 |
| 2023-02-24 04:00:00 | 26045.4 |
| 2023-02-24 05:00:00 | 27773.1 |
| 2023-02-24 06:00:00 | 29804.7 |
| 2023-02-24 07:00:00 | 31483.9 |
| 2023-02-24 08:00:00 | 32447   |
| 2023-02-24 09:00:00 | 32819   |
| 2023-02-24 10:00:00 | 32970.4 |
| 2023-02-24 11:00:00 | 33098.5 |
| 2023-02-24 12:00:00 | 33064   |
| 2023-02-24 13:00:00 | 32644.8 |
| 2023-02-24 14:00:00 | 31925.6 |
| 2023-02-24 15:00:00 | 31379.3 |
| 2023-02-24 16:00:00 | 31499   |
| 2023-02-24 17:00:00 | 32298.6 |
| 2023-02-24 18:00:00 | 33173.7 |
| 2023-02-24 19:00:00 | 33299.9 |
| 2023-02-24 20:00:00 | 32246   |
| 2023-02-24 21:00:00 | 30273.3 |
| 2023-02-24 22:00:00 | 28081.7 |
| 2023-02-24 23:00:00 | 26265.2 |


<div style="text-align: right"><font size=4> <i class="fa fa-check-square-o" aria-hidden="true" style="color:#00586D"></i></font></div>

--- 

Con esta función ya podemos evaluar nuestro modelo. En este caso, debemos recordar que debemos evaluar nuestro modelo de forma diaria, ya que nuestro objetivo es hacer pronósticos a un día vista. Para ello podemos valernos de la función anterior y recorrer diariamente los valores de un conjunto de test, añadiendo en el training los valores hasta la fecha. Solo usaremos los primeros días del mes de enero para probar el método. Veamos como hacerlo. 

In [37]:
test_dates = list(pd.date_range(start=datetime.datetime(2023, 1, 1), end=datetime.datetime(2023, 1, 10), freq='D'))

In [38]:
maes = []
for test_date in test_dates:
    training = df[:test_date-datetime.timedelta(minutes=1)]
    test = df[test_date:test_date+datetime.timedelta(days=1, minutes=-1)]
    result = train_forecast(df, test.index)
    r = mean_absolute_error(test, result)
    maes.append(r)

13:29:43 - cmdstanpy - INFO - Chain [1] start processing
13:29:50 - cmdstanpy - INFO - Chain [1] done processing
13:29:50 - cmdstanpy - INFO - Chain [1] start processing
13:29:56 - cmdstanpy - INFO - Chain [1] done processing
13:29:57 - cmdstanpy - INFO - Chain [1] start processing
13:30:03 - cmdstanpy - INFO - Chain [1] done processing
13:30:03 - cmdstanpy - INFO - Chain [1] start processing
13:30:10 - cmdstanpy - INFO - Chain [1] done processing
13:30:10 - cmdstanpy - INFO - Chain [1] start processing
13:30:16 - cmdstanpy - INFO - Chain [1] done processing
13:30:16 - cmdstanpy - INFO - Chain [1] start processing
13:30:22 - cmdstanpy - INFO - Chain [1] done processing
13:30:23 - cmdstanpy - INFO - Chain [1] start processing
13:30:29 - cmdstanpy - INFO - Chain [1] done processing
13:30:29 - cmdstanpy - INFO - Chain [1] start processing
13:30:36 - cmdstanpy - INFO - Chain [1] done processing
13:30:36 - cmdstanpy - INFO - Chain [1] start processing
13:30:43 - cmdstanpy - INFO - Chain [1]

In [41]:
np.mean(maes)

2416.8888287164104

Con este modelo ya podemos añadir la información de pronóstico como un nuevo campo en nuestra serie en InfluxDB. Para ello seguiremos la siguiente estrategia:
- Nos centraremos solo en el último més de datos disponible. Para el último día, que posiblemente no estará completo, deberemos añadir las horas que falten. 
- Generaremos un pronóstico para cada día usando el modelo que creamos anteriormente. 
- Subiremos este pronóstico directamente a InfluxDB

Implementaremos este proceso por partes a continuación. 


--- 

### <font color="#004D7F"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#004D7F"></i> Tarea 3 </font>
<br>

Implementa la siguiente función que, dada una fecha, devuelve una lista con los timestamps ordenados de sus horas. 

In [42]:
def get_hours(date: datetime.datetime) -> list :
    ######## Solución
    ###

In [43]:
get_hours(datetime.datetime(2023, 2, 28))

[Timestamp('2023-02-28 00:00:00', freq='H'),
 Timestamp('2023-02-28 01:00:00', freq='H'),
 Timestamp('2023-02-28 02:00:00', freq='H'),
 Timestamp('2023-02-28 03:00:00', freq='H'),
 Timestamp('2023-02-28 04:00:00', freq='H'),
 Timestamp('2023-02-28 05:00:00', freq='H'),
 Timestamp('2023-02-28 06:00:00', freq='H'),
 Timestamp('2023-02-28 07:00:00', freq='H'),
 Timestamp('2023-02-28 08:00:00', freq='H'),
 Timestamp('2023-02-28 09:00:00', freq='H'),
 Timestamp('2023-02-28 10:00:00', freq='H'),
 Timestamp('2023-02-28 11:00:00', freq='H'),
 Timestamp('2023-02-28 12:00:00', freq='H'),
 Timestamp('2023-02-28 13:00:00', freq='H'),
 Timestamp('2023-02-28 14:00:00', freq='H'),
 Timestamp('2023-02-28 15:00:00', freq='H'),
 Timestamp('2023-02-28 16:00:00', freq='H'),
 Timestamp('2023-02-28 17:00:00', freq='H'),
 Timestamp('2023-02-28 18:00:00', freq='H'),
 Timestamp('2023-02-28 19:00:00', freq='H'),
 Timestamp('2023-02-28 20:00:00', freq='H'),
 Timestamp('2023-02-28 21:00:00', freq='H'),
 Timestamp

Resutado esperado:
```
[Timestamp('2023-02-28 00:00:00', freq='H'),
 Timestamp('2023-02-28 01:00:00', freq='H'),
 Timestamp('2023-02-28 02:00:00', freq='H'),
 Timestamp('2023-02-28 03:00:00', freq='H'),
 Timestamp('2023-02-28 04:00:00', freq='H'),
 Timestamp('2023-02-28 05:00:00', freq='H'),
 Timestamp('2023-02-28 06:00:00', freq='H'),
 Timestamp('2023-02-28 07:00:00', freq='H'),
 Timestamp('2023-02-28 08:00:00', freq='H'),
 Timestamp('2023-02-28 09:00:00', freq='H'),
 Timestamp('2023-02-28 10:00:00', freq='H'),
 Timestamp('2023-02-28 11:00:00', freq='H'),
 Timestamp('2023-02-28 12:00:00', freq='H'),
 Timestamp('2023-02-28 13:00:00', freq='H'),
 Timestamp('2023-02-28 14:00:00', freq='H'),
 Timestamp('2023-02-28 15:00:00', freq='H'),
 Timestamp('2023-02-28 16:00:00', freq='H'),
 Timestamp('2023-02-28 17:00:00', freq='H'),
 Timestamp('2023-02-28 18:00:00', freq='H'),
 Timestamp('2023-02-28 19:00:00', freq='H'),
 Timestamp('2023-02-28 20:00:00', freq='H'),
 Timestamp('2023-02-28 21:00:00', freq='H'),
 Timestamp('2023-02-28 22:00:00', freq='H'),
 Timestamp('2023-02-28 23:00:00', freq='H')]
```

<div style="text-align: right"><font size=4> <i class="fa fa-check-square-o" aria-hidden="true" style="color:#00586D"></i></font></div>

--- 

Para ingestar el pronóstico en InfluxDB, podemos usar la siguiente función, similar a la función que utilizásteis en la primera práctica y que básicamente toma el pronóstico del modelo de Prophet y lo ingesta usando la API de escritura de InfluxDB. El pronóstico se ingestará en la misma medida que la demanda, pero con el campo `CP7Forecast`.

In [44]:
def save_to_influxdb(df: pd.DataFrame, client: InfluxDBClient, bucket: str, org: str) -> pd.DataFrame:
    """
    Escribe el pronóstico en InfluxDB
    """
    df = df.reset_index()
    df['time'] = df['datetime']
    df['CP7Forecast'] = df['yhat']
    to_write = df[['time', 'CP7Forecast']]
    to_write = to_write.set_index('time')[['CP7Forecast']]
    write_api = client.write_api(write_options=SYNCHRONOUS)
    write_api.write(bucket, org, record=to_write, data_frame_measurement_name='demand')

Con estas funciones ya tenemos todo lo necesario para poder escribir en InfluxDB los datos de pronóstico. Para ello, partiendo de un día inicial desde el que empezaremos a generar pronósticos, usaremos una estrategia similar al método de evaluación anterior, simplemente teniendo en cuenta que generaremos nosotros las horas en vez de usar las que aparecen en un conjunto de test. A continuación mostramos como ingestar los resultados a partir de un día determinado. Podéis ejecutar el código a continuación para añadir el pronóstico a vuestra instancia de InfluxDB.

In [45]:
begin = datetime.datetime(2023, 2, 22)
dates_to_ingest = pd.date_range(start=begin, end=datetime.datetime.now().date(), freq='D')
dates_to_ingest

DatetimeIndex(['2023-02-22', '2023-02-23', '2023-02-24', '2023-02-25',
               '2023-02-26'],
              dtype='datetime64[ns]', freq='D')

In [46]:
for dt in dates_to_ingest:
    hours = get_hours(dt)
    print(f"Ingesting for date: {dt}")
    training = df[:dt-datetime.timedelta(minutes=1)]
    result = train_forecast(df, hours)
    save_to_influxdb(result, client, bucket, org)

Ingesting for date: 2023-02-22 00:00:00


13:31:33 - cmdstanpy - INFO - Chain [1] start processing
13:31:39 - cmdstanpy - INFO - Chain [1] done processing


Ingesting for date: 2023-02-23 00:00:00


13:31:39 - cmdstanpy - INFO - Chain [1] start processing
13:31:45 - cmdstanpy - INFO - Chain [1] done processing


Ingesting for date: 2023-02-24 00:00:00


13:31:45 - cmdstanpy - INFO - Chain [1] start processing
13:31:51 - cmdstanpy - INFO - Chain [1] done processing


Ingesting for date: 2023-02-25 00:00:00


13:31:52 - cmdstanpy - INFO - Chain [1] start processing
13:31:58 - cmdstanpy - INFO - Chain [1] done processing


Ingesting for date: 2023-02-26 00:00:00


13:31:58 - cmdstanpy - INFO - Chain [1] start processing
13:32:05 - cmdstanpy - INFO - Chain [1] done processing


Cuando acabe el proceso anterior ya deberíais tener los resultados en InfluxDB. Solo quedaría adaptar el dashboard anterior para visualizar también los datos de pronóstico. 


--- 

### <font color="#004D7F"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#004D7F"></i> Tarea 4</font>
<br>

Añade a la celda tipo graph del dashboard que creaste en la primera parte del capstone la serie predicha por el modelo que hemos entrenado.

Para visualizar el pronóstico a futuro, necesitarás modificar el filtro de tiempo para incluir el día en curso. 

**Añade una captura de tu dashboard a continuación (incluyéndo el dashboard con la entrega)**. 


Resultado esperado (aprox):

![tarea4](img/p2tarea4.png)

<div style="text-align: right"><font size=4> <i class="fa fa-check-square-o" aria-hidden="true" style="color:#00586D"></i></font></div>

--- 